In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os
from torch.utils import data
from wavenet import Wavenet
from transformData import x_mu_law_encode,y_mu_law_encode,mu_law_decode,onehot,cateToSignal
from readDataset import Dataset

In [2]:
sampleSize=32000#the length of the sample size
quantization_channels=256
sample_rate=16000
dilations=[2**i for i in range(9)]*5  #idea from wavenet, have more receptive field
residualDim=128 #
skipDim=512
shapeoftest = 190500
filterSize=3
resumefile='./model/testac' # name of checkpoint
lossname='testacloss.txt' # name of loss file
continueTrain=True # whether use checkpoint
pad = np.sum(dilations) # padding for dilate convolutional layers
lossrecord=[]  #list for record loss
#pad=0

    #            |----------------------------------------|     *residual*
    #            |                                        |
    #            |    |-- conv -- tanh --|                |
    # -> dilate -|----|                  * ----|-- 1x1 -- + -->	*input*
    #                 |-- conv -- sigm --|     |    ||
    #                                         1x1=residualDim
    #                                          |
    # ---------------------------------------> + ------------->	*skip=skipDim*
    image changed from https://github.com/vincentherrmann/pytorch-wavenet/blob/master/wavenet_model.py

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use specific GPU

In [4]:
use_cuda = torch.cuda.is_available() # whether have available GPU
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor') #set_default_tensor_type as cuda tensor
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} 

In [5]:
params = {'batch_size': 1,'shuffle': True,'num_workers': 1}
training_set = Dataset(['origin_mix'],['origin_vocal'],'./vsCorpus/','./vsCorpus/')
testing_set = Dataset(['pred_mix'],['pred_mix'],'./vsCorpus/','./vsCorpus/')
loadtr = data.DataLoader(training_set, **params) #pytorch dataloader, more faster than mine
loadval = data.DataLoader(testing_set, **params)

In [6]:
model = Wavenet(pad,skipDim,quantization_channels,residualDim,dilations).cuda()
criterion = nn.CrossEntropyLoss()
#in wavenet paper, they said crossentropyloss is far better than MSELoss
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
#use adam to train
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

In [7]:
if continueTrain:# if continueTrain, the program will find the checkpoints
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

=> loading checkpoint './model/testac'
=> loaded checkpoint './model/testac' (epoch 65)


In [ ]:
def val(xtrain,ytrain): #validation last 15 seconds of the audio.
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        idx = np.arange(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
        print(correct,'accurate')
        print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-start_time))
        
        listofpred = []
        for ind in range(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,sampleSize):
            output = model(xtrain[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/notexval.wav', ans, sample_rate)
        print('val stored done',time.time() - start_time)
        

def test(xtrain):# testing data
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for iloader,(xtest,_) in enumerate(loadval):
            listofpred = []
            for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
                output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/notexte.wav', ans, sample_rate)

            listofpred=[]
            for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
                output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/notextr.wav', ans, sample_rate)
            print('test stored done',time.time() - start_time)
    
def train(epoch):#training data, the audio except for last 15 seconds
    model.train()
    for iloader,(xtrain,ytrain) in enumerate(loadtr):
        idx = np.arange(pad,xtrain.shape[-1]-pad-11*sampleSize,16000)
        np.random.shuffle(idx)#random the starting points
        for i, ind in enumerate(idx):
            start_time = time.time()
            data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
            output = model(data)
            loss = criterion(output, target)
            lossrecord.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                    epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
            if i % 100 == 0:
                with open("./lossRecord/"+lossname, "w") as f:
                    for s in lossrecord:
                        f.write(str(s) +"\n")
                print('write finish')

        val(xtrain,ytrain)
        test(xtrain)
        state={'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}
        torch.save(state, resumefile)

In [ ]:
for epoch in range(100000):
    train(epoch)

Train Epoch: 0 [0/214 (0%)] Loss:1.166864: , (2.126 sec/step)
write finish
Train Epoch: 0 [1/214 (0%)] Loss:1.355406: , (2.119 sec/step)
Train Epoch: 0 [2/214 (1%)] Loss:1.352069: , (2.118 sec/step)
Train Epoch: 0 [3/214 (1%)] Loss:1.534732: , (2.119 sec/step)
Train Epoch: 0 [4/214 (2%)] Loss:1.561106: , (2.121 sec/step)
Train Epoch: 0 [5/214 (2%)] Loss:1.463008: , (2.120 sec/step)
Train Epoch: 0 [6/214 (3%)] Loss:1.260574: , (2.120 sec/step)
Train Epoch: 0 [7/214 (3%)] Loss:1.250476: , (2.120 sec/step)
Train Epoch: 0 [8/214 (4%)] Loss:1.483780: , (2.119 sec/step)
Train Epoch: 0 [9/214 (4%)] Loss:1.308076: , (2.122 sec/step)
Train Epoch: 0 [10/214 (5%)] Loss:1.519015: , (2.125 sec/step)
Train Epoch: 0 [11/214 (5%)] Loss:1.315021: , (2.126 sec/step)
Train Epoch: 0 [12/214 (6%)] Loss:1.489690: , (2.126 sec/step)
Train Epoch: 0 [13/214 (6%)] Loss:1.509562: , (2.126 sec/step)
Train Epoch: 0 [14/214 (7%)] Loss:1.503773: , (2.126 sec/step)
Train Epoch: 0 [15/214 (7%)] Loss:1.525444: , (2.127

Train Epoch: 0 [128/214 (60%)] Loss:1.300221: , (2.135 sec/step)
Train Epoch: 0 [129/214 (60%)] Loss:1.816449: , (2.133 sec/step)
Train Epoch: 0 [130/214 (61%)] Loss:1.314174: , (2.132 sec/step)
Train Epoch: 0 [131/214 (61%)] Loss:1.733001: , (2.135 sec/step)
Train Epoch: 0 [132/214 (62%)] Loss:1.506857: , (2.134 sec/step)
Train Epoch: 0 [133/214 (62%)] Loss:1.556709: , (2.134 sec/step)
Train Epoch: 0 [134/214 (63%)] Loss:1.845950: , (2.135 sec/step)
Train Epoch: 0 [135/214 (63%)] Loss:1.729309: , (2.137 sec/step)
Train Epoch: 0 [136/214 (64%)] Loss:1.774106: , (2.134 sec/step)
Train Epoch: 0 [137/214 (64%)] Loss:1.488728: , (2.134 sec/step)
Train Epoch: 0 [138/214 (64%)] Loss:1.771289: , (2.134 sec/step)
Train Epoch: 0 [139/214 (65%)] Loss:1.577339: , (2.134 sec/step)
Train Epoch: 0 [140/214 (65%)] Loss:1.515217: , (2.135 sec/step)
Train Epoch: 0 [141/214 (66%)] Loss:1.901796: , (2.135 sec/step)
Train Epoch: 0 [142/214 (66%)] Loss:2.187353: , (2.135 sec/step)
Train Epoch: 0 [143/214 (

Train Epoch: 1 [39/214 (18%)] Loss:1.459196: , (2.133 sec/step)
Train Epoch: 1 [40/214 (19%)] Loss:1.497208: , (2.134 sec/step)
Train Epoch: 1 [41/214 (19%)] Loss:1.382822: , (2.135 sec/step)
Train Epoch: 1 [42/214 (20%)] Loss:2.111525: , (2.133 sec/step)
Train Epoch: 1 [43/214 (20%)] Loss:1.923267: , (2.135 sec/step)
Train Epoch: 1 [44/214 (21%)] Loss:1.563558: , (2.134 sec/step)
Train Epoch: 1 [45/214 (21%)] Loss:1.378688: , (2.135 sec/step)
Train Epoch: 1 [46/214 (21%)] Loss:1.312755: , (2.134 sec/step)
Train Epoch: 1 [47/214 (22%)] Loss:1.369437: , (2.135 sec/step)
Train Epoch: 1 [48/214 (22%)] Loss:1.508787: , (2.134 sec/step)
Train Epoch: 1 [49/214 (23%)] Loss:1.234402: , (2.133 sec/step)
Train Epoch: 1 [50/214 (23%)] Loss:1.740749: , (2.133 sec/step)
Train Epoch: 1 [51/214 (24%)] Loss:1.273612: , (2.134 sec/step)
Train Epoch: 1 [52/214 (24%)] Loss:1.409329: , (2.134 sec/step)
Train Epoch: 1 [53/214 (25%)] Loss:1.700603: , (2.135 sec/step)
Train Epoch: 1 [54/214 (25%)] Loss:1.482

Train Epoch: 1 [166/214 (78%)] Loss:1.407983: , (2.133 sec/step)
Train Epoch: 1 [167/214 (78%)] Loss:1.260544: , (2.135 sec/step)
Train Epoch: 1 [168/214 (79%)] Loss:1.449898: , (2.135 sec/step)
Train Epoch: 1 [169/214 (79%)] Loss:1.127281: , (2.133 sec/step)
Train Epoch: 1 [170/214 (79%)] Loss:1.354030: , (2.135 sec/step)
Train Epoch: 1 [171/214 (80%)] Loss:1.673305: , (2.134 sec/step)
Train Epoch: 1 [172/214 (80%)] Loss:1.544323: , (2.133 sec/step)
Train Epoch: 1 [173/214 (81%)] Loss:2.015117: , (2.134 sec/step)
Train Epoch: 1 [174/214 (81%)] Loss:2.445705: , (2.135 sec/step)
Train Epoch: 1 [175/214 (82%)] Loss:1.610141: , (2.132 sec/step)
Train Epoch: 1 [176/214 (82%)] Loss:1.429478: , (2.134 sec/step)
Train Epoch: 1 [177/214 (83%)] Loss:2.193564: , (2.134 sec/step)
Train Epoch: 1 [178/214 (83%)] Loss:1.807058: , (2.134 sec/step)
Train Epoch: 1 [179/214 (84%)] Loss:1.321150: , (2.133 sec/step)
Train Epoch: 1 [180/214 (84%)] Loss:1.421357: , (2.132 sec/step)
Train Epoch: 1 [181/214 (